## **Instalação do pacote transformers**

In [ ]:
!pip install transformers

## **Importação de bibliotecas**

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import re
from transformers import pipeline
import torch
from transformers import AutoTokenizer
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads

## **Carregamento do modelo BERTimbau**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased', do_lower_case=False)
model = AutoModel.from_pretrained('neuralmind/bert-large-portuguese-cased')

## **Carregamento dos conjuntos de dados**

In [ ]:
resp = pd.read_csv('/content/drive/MyDrive/amostra_recurso.csv')
temas_repetitivos = pd.read_csv('/content/drive/MyDrive/temas.csv', sep=';',encoding='ISO-8859-1' )
dicionario_temas = pd.read_csv('/content/drive/MyDrive/dicionario-temas.csv', sep=';')

## **Criação de data frame de temas contendo apenas numeração e texto**

In [ ]:
temas = temas_repetitivos[['numeroPrecedente','questaoSubmetidaAJulgamento']].copy()
temas.columns = ['numTema','texto']
temas.head()

,numTema,texto
0,1,Validade da cláusula contratual de plano de sa...
1,2,"Definir se a Lei n. 13.000/2014, que assegurou..."
2,3,Discute-se o termo inicial do prazo prescricio...
3,4,Aplicação do inciso II do art. 115 da Lei 8.21...
4,5,Aplicabilidade da Súmula 345 do STJ diante da ...


#**Cria embedding a partir de texto**

---
Os modelos do projeto transformers geram embeddings de tamanho fixo para cada palavra do texto ( além de embeddings para tokens especiais). Então os embeddings foram agrupados pelo cálculo de média pra gerar um único embedding que represente todo o texto.




In [ ]:
def create_embedding(text):
  input_ids = tokenizer.encode(text, return_tensors='pt')

  with torch.no_grad():
      outs = model(input_ids)
      encoded = outs[0][0, 1:-1]
      #cálculo da média
      mean_embedding = torch.mean(encoded, dim=0)
      return mean_embedding



##**Gera embeddings de todos os temas**
Salva em arquivo

In [48]:
lista_embeddings_temas = []
for indice, linha in temas.iterrows():
    try:
      embedding_tema = create_embedding(linha['texto'])
      tupla_numTema_embedding = (linha['numTema'],embedding_tema)
      lista_embeddings_temas.append(tupla_numTema_embedding)
    except Exception as erro:
      print(f"Erro no indice {indice} : {erro}")
torch.save(lista_embeddings_temas, 'embeddings_temas.pt')


Erro no indice 101 : The expanded size of the tensor (932) must match the existing size (512) at non-singleton dimension 1.  Target sizes: [1, 932].  Tensor sizes: [1, 512]


## **Carregamento do arquivo que contem os embeddings dos temas**
Para acelerar a execução da avaliação de similaridade, foi gerado um arquivo que contém a numeração e o word embedding de cada tema

In [49]:
embeddings_temas = torch.load('/content/drive/MyDrive/embeddings_temas.pt')

#**Lê recurso**

In [50]:
def read_text(dataframe, linha, coluna):
    texto = dataframe.at[linha, coluna]
    return texto

#**Identifica fim do paragrafo**

A implementar

#**Extrai fragmentos do texto**
Necessário devido à limitação de tamanho imposta pelo modelo

In [51]:
def extract_fragments(text, max_words_per_fragment):
    words = text.split()  # Divide o texto em palavras
    fragments = []
    current_fragment = []

    for word in words:
        if len(current_fragment) + len(word.split()) <= max_words_per_fragment:
            current_fragment.append(word)
        else:
            fragments.append(" ".join(current_fragment))
            current_fragment = [word]

    # Adiciona o último fragmento, se houver
    if current_fragment:
        fragments.append(" ".join(current_fragment))

    return fragments


## **Ordena lista de tuplas**
Ordena pelo segundo elemento em ordem descendente

In [52]:
def sort_list(lista):
	return(sorted(lista, key = lambda x: x[1],reverse=True))


## **Início do processamento**

In [ ]:
# Texto
text = read_text(resp,1,'recurso')

tensores_fragmentos = []

# Quantidade máxima de palavras por fragmento
max_words_per_fragment = 170

# Extrai os fragmentos
fragments = extract_fragments(text, max_words_per_fragment)

# Imprime os fragmentos
for idx, fragment in enumerate(fragments, start=1):
    print(f"Fragmento {idx}: {fragment}")

#Cria embeddings
for idx, fragment in enumerate(fragments, start=1):
    print(f"Criando embedding do fragmento {idx} ... ")
    tensor_fragmento = create_embedding(fragment)
    tensores_fragmentos.append(tensor_fragmento)

print("Tensores fragmentos")
print(tensores_fragmentos)
tensor_final = torch.stack(tensores_fragmentos)

print("Tensor final")
print(tensor_final)
tensor_final_media = torch.mean(tensor_final, dim=0)

#Imprime Embedding do texto completo
print("Embedding do texto completo a partir da média")
print(tensor_final_media)
embedding_resp = tensor_final_media

Fragmento 1: ADVOCACIA-GERAL DA UNIÃO PROCURADORIA-GERAL FEDERAL PROCURADORIA REGIONAL FEDERAL 2a REGIÃO NPREV - SUBNÚCLEO FINALÍSTICO ATUAÇÃO TÉCNICA ESPECIALIZADA PREVIDENCIÁRIA TRIBUNAIS - EATE/TRF2/PREV AV. NILO PEÇANHA, 151 - CENTRO, RIO DE JANEIRO/RJ, CEP 20020-100 EXCELENTÍSSIMO(A) SENHOR(A) DESEMBARGADOR(A) DO(A) GABINETE 03 NÚMERO: 0216608-09.2017.4.02.5107 RECORRENTE(S): INSTITUTO NACIONAL DO SEGURO SOCIAL - INSS RECORRIDO(S): GESSE JAMES ELOY SANTOS INSTITUTO NACIONAL DO SEGURO SOCIAL - INSS, pessoa jurídica de direito público, representado(a) pelo membro da Advocacia-Geral da União infra assinado(a), vem, respeitosamente, à presença de Vossa Excelência, interpor, com fulcro no art. 105, inciso III, alínea 'a' da Constituição Federal da República Federativa do Brasil, vem interpor RECURSO ESPECIAL pelas razões que seguem, em anexo, requerendo o seu recebimento, processamento e remessa ao Colendo Superior Tribunal de Justiça, na forma da lei. RAZÓES DO RECURSO ESPECIAL DA TEM

## **Calcula Similaridade entre texto do recurso especial e cada um dos temas**

In [53]:
lista_similaridade = []

for indice, tupla_tema in enumerate(embeddings_temas):
    cos = nn.CosineSimilarity(dim=0, eps=1e-6)
    #primeiro elemento da tupla contem o número do tem e o segundo o texto do tema
    similaridade = cos(embedding_resp,tupla_tema[1])
    tupla = (tupla_tema[0],similaridade)
    lista_similaridade.append(tupla)

print(sort_list(lista_similaridade))



[(1090, tensor(0.9707)), (274, tensor(0.9706)), (175, tensor(0.9578)), (149, tensor(0.9557)), (1150, tensor(0.9556)), (247, tensor(0.9552)), (368, tensor(0.9549)), (126, tensor(0.9543)), (493, tensor(0.9534)), (516, tensor(0.9524)), (9, tensor(0.9519)), (401, tensor(0.9517)), (351, tensor(0.9514)), (359, tensor(0.9506)), (531, tensor(0.9499)), (524, tensor(0.9497)), (165, tensor(0.9497)), (259, tensor(0.9495)), (611, tensor(0.9489)), (1142, tensor(0.9477)), (996, tensor(0.9474)), (7, tensor(0.9472)), (694, tensor(0.9465)), (435, tensor(0.9463)), (105, tensor(0.9462)), (337, tensor(0.9461)), (47, tensor(0.9457)), (216, tensor(0.9457)), (362, tensor(0.9454)), (1147, tensor(0.9448)), (974, tensor(0.9444)), (931, tensor(0.9442)), (10, tensor(0.9441)), (203, tensor(0.9439)), (292, tensor(0.9439)), (360, tensor(0.9438)), (100, tensor(0.9436)), (416, tensor(0.9434)), (143, tensor(0.9434)), (954, tensor(0.9433)), (485, tensor(0.9430)), (419, tensor(0.9425)), (174, tensor(0.9425)), (723, tensor

In [ ]:
print(dicionario_temas)

                          Campo                                 Nome extenso  \
0          sequencialPrecedente  Número sequencial do precedente qualificado   
1                tipoPrecedente               Tipo de precedente qualificado   
2              numeroPrecedente             Número do precedente qualificado   
3          dataPrimeiraAfetacao                    Data da primeira afetação   
4                dataJulgamento                           Data de julgamento   
5                      situacao                                     Situação   
6     informacoesComplementares                   Informações complementares   
7   questaoSubmetidaAJulgamento               Questão submetida a julgamento   
8                   teseFirmada                                 Tese firmada   
9             anotacoesNUGEPNAC                           Anotações NUGEPNAC   
10           delimitacaoJulgado                       Delimitação do julgado   
11         entendimentoAnterior         